In [ ]:
import datetime
ohlc_ticker = 'GME'
oi_ticker = 'GME'
startdate = '2021-12-15'
import pandas_ta as pta

In [ ]:
import pandas as pd
import datetime
from jinja2 import Template
import yfinance as yf
import csv
from plotly.subplots import make_subplots
import plotly.graph_objects as go

timeframe = '1d'
enddate = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

In [ ]:

from unittest.mock import call


oi_file_suffix = '_oi_data.csv'
banner = 'Price: ' + ohlc_ticker + ' / OI: ' + oi_ticker

ndoi_combo = pd.DataFrame()

def fig_init():
    # fig = make_subplots(specs=[[{"secondary_y": True,"r":0.06}]])
    fig.data = []
    fig.layout.shapes = []
    # fig.layout = {}

def load_oi(ticker):
    oi_file = ticker + oi_file_suffix
    df_oi = pd.read_csv(oi_file,parse_dates=['data.date'])
    df_oi = df_oi[~(df_oi['data.date'] < startdate)]
    if ticker == 'GME': # for some reason yfinance doesn't report the splut stuff appropriately so it's hard coded here; you can replicate for other tickers
        split_date = '2022-07-22'
        split_factor = 4
        df_oi = split_adjust_oi(df_oi,split_date,split_factor)
    return df_oi

def split_adjust_oi(df_oi,split_date,split_factor):
    df_oi.loc[df_oi['data.date'] <= split_date, ['oi.calls','oi.puts','oi.short.calls','oi.long.calls','oi.short.puts','oi.long.puts']] *= split_factor
    return df_oi

def trace_candle(df_ohlc):
    fig.add_trace(
        go.Candlestick(x=df_ohlc.index, open=df_ohlc['Open'], high=df_ohlc['High'], low=df_ohlc['Low'], close=df_ohlc['Close'],name="Price"),
        secondary_y=False
    )
    
def trace_calls(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'], y=df_oi['oi.calls'], name="Call OI"),
        secondary_y=True
    )

def trace_puts(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['oi.puts'], name="Put OI"),
        secondary_y=True
    )

def trace_short_calls(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['oi.short.calls'],name="Dealer Short Call OI"),
        secondary_y=True
    )

def trace_long_calls(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['oi.long.calls'],name="Dealer Long Call OI"),
        secondary_y=True
    )

def trace_short_puts(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['oi.short.puts'],name="Dealer Short Put OI"),
        secondary_y=True
    )

def trace_long_puts(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['oi.long.puts'], name="Dealer Long Put OI"),
        secondary_y=True
    )

def trace_voex_roc(df_oi):
    df_oi['vx-ROC'] = pta.roc(df_oi['voex'],10)
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['vx-ROC'],name="VoEx Rate of Change"), 
        secondary_y=True
    )
    # fig.add_shape(type='line',line=dict(color=))
def trace_voex_rsi(df_oi):
    df_oi['vx-RSI'] = pta.rsi(df_oi['voex'],length=14)
    df_oi['vx-RSI-MA'] = df_oi['vx-RSI'].rolling(14).mean()
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['vx-RSI'], name="VoEx RSI"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['vx-RSI-MA'], name="VoEx RSI SMA"),
        secondary_y=True
    )
    
    fig.add_shape(type='line',line=dict(color='Red'),xref='x domain',yref='y2',x0=0,x1=1,y0=40,y1=40,secondary_y=True)
    fig.add_shape(type='line',line=dict(color='Red',dash='dash'),xref='x domain',yref='y2',x0=0,x1=1,y0=45,y1=45,secondary_y=True)
    fig.add_shape(type='line',line=dict(color='Green',dash='dash'),xref='x domain',yref='y2',x0=0,x1=1,y0=55,y1=55,secondary_y=True)
    fig.add_shape(type='line',line=dict(color='Green'),xref='x domain',yref='y2',x0=0,x1=1,y0=60,y1=60,secondary_y=True)
    
def trace_voex(df_oi):
    df_oi['vx-EMA5'] = df_oi.voex.ewm(span=5, adjust=False).mean()
    df_oi['vx-EMA10'] = df_oi.voex.ewm(span=10, adjust=False).mean()
    df_oi['vx-EMA20'] = df_oi.voex.ewm(span=20, adjust=False).mean()
    df_oi['vx-MA5'] = df_oi.voex.rolling(5).mean()
    df_oi['vx-MA10'] = df_oi.voex.rolling(10).mean()
    df_oi['vx-MA20'] = df_oi.voex.rolling(20).mean()

    # fig.add_shape(type='line',line=dict(color='Gray',),xref='x domain',yref='y2',x0=0,x1=1,y0=0,y1=0)

    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['voex'], name="VoEx"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['vx-MA5'], name="VoEx 5ma"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['vx-MA10'], name="VoEx 10ma"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['vx-MA20'], name="VoEx 20ma"),
        secondary_y=True
    )


def trace_iv(df_oi):
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['iv'], name="IV"),
        secondary_y=True
    )

def trace_bull_oi(df_oi):
    df_oi['bull_oi'] = df_oi['oi.short.calls'] + df_oi['oi.long.puts']
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['bull_oi'],name="Net Bullish OI (dealer short call + long put)"),
        secondary_y=True
    )

def trace_bear_oi(df_oi):
    df_oi['bear_oi'] = df_oi['oi.long.calls'] + df_oi['oi.short.puts']
    fig.add_trace(
        go.Scatter(x=df_oi['data.date'],y=df_oi['bear_oi'],name="Net Bearish OI (dealer long call + short put)"),
        secondary_y=True
    )

def trace_call_skew(df_ndoi):
    fig.add_shape(type='line',line=dict(color='Gray',),xref='x domain',yref='y2',x0=0,x1=1,y0=0,y1=0)
    fig.add_trace(
        go.Scatter(x=df_ndoi['data.date'],y=df_ndoi['call_skew'], name="Directional Call Skew"),
        secondary_y=True
    )
def trace_put_skew(df_ndoi):
    fig.add_shape(type='line',line=dict(color='Gray',),xref='x domain',yref='y2',x0=0,x1=1,y0=0,y1=0)
    fig.add_trace(
        go.Scatter(x=df_ndoi['data.date'],y=df_ndoi['put_skew'], name="Directional Put Skew"),
        secondary_y=True
    ) 
    
def trace_ndoi(df_ndoi):
    df_ndoi['MA5'] = df_ndoi.ndoi.rolling(5).mean()
    df_ndoi['MA10'] = df_ndoi.ndoi.rolling(10).mean()
    df_ndoi['MA20'] = df_ndoi.ndoi.rolling(20).mean()
    fig.add_shape(type='line',line=dict(color='Gray',),xref='x domain',yref='y2',x0=0,x1=1,y0=0,y1=0)
    fig.add_trace(
        go.Scatter(x=df_ndoi['data.date'],y=df_ndoi['ndoi'], name="Net Directional OI"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_ndoi['data.date'],y=df_ndoi['MA5'], name="ndoi5ma"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_ndoi['data.date'],y=df_ndoi['MA10'], name="ndoi10ma"),
        secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=df_ndoi['data.date'],y=df_ndoi['MA20'], name="ndoi20ma"),
        secondary_y=True
    )   

def update_axes(title,ytitle):
    fig.update_xaxes(title_text="Date")
    fig.update_xaxes(rangebreaks=[
        dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        dict(values=["2022-05-30","2022-06-20","2022-09-05"])
    ])
    fig.update_layout(
        title_text=title,
        xaxis=dict(rangeslider=dict(visible=False))
    )
    fig.layout.xaxis.automargin = True
    fig.update_yaxes(title_text="Price", secondary_y=False)
    fig.update_yaxes(title_text=ytitle, secondary_y=True)

def fig_show(title,ytitle="OI"):
    update_axes(title=title,ytitle=ytitle)
    fig.show()

def calculate_ndoi(df_oi):
    #df_oi =df_oi.reset_index() # ensure indexes pair with number of rows
    df_ndoi = pd.DataFrame()
    df_ndoi_temp = pd.DataFrame()
    for index, row in df_oi.iterrows():
        date = pd.to_datetime(row['data.date'])
        # try:
        bulloi = int(row['oi.long.puts']) + int(row['oi.short.calls'])
        bearoi = int(row['oi.short.puts']) + int(row['oi.long.calls'])
        call_skew = int(row['oi.short.calls']) - int(row['oi.long.calls']) 
        put_skew = int(row['oi.long.puts']) - int(row['oi.short.puts'])
        ndoi = bulloi - bearoi
        df_ndoi_temp = pd.DataFrame({'data.date': date, 'ndoi': ndoi, 'call_skew': call_skew, 'put_skew': put_skew},index=[0])
        df_ndoi = pd.concat([df_ndoi, df_ndoi_temp])
        # except TypeError:
        #     pass
    return df_ndoi

def generate_combo_ndoi(ticker_list,use_modifier=True):
    ndoi_combo = pd.DataFrame()
    for ticker,factor in ticker_list.items():
        df_oi_temp = load_oi(ticker)
        df_ndoi_temp = calculate_ndoi(df_oi_temp)
        if use_modifier:
            df_ndoi_temp['ndoi'] = df_ndoi_temp['ndoi'] * factor * 100
            
        if ndoi_combo.empty:
            ndoi_combo = df_ndoi_temp.copy(deep=True)
        else:
            ndoi_combo['ndoi'] = ndoi_combo['ndoi'] + df_ndoi_temp['ndoi']
    return ndoi_combo

df_oi = load_oi(oi_ticker)
df_ohlc = yf.download(tickers=ohlc_ticker,interval=timeframe,start=startdate,end=enddate,rounding=True)

fig = make_subplots(specs=[[{"secondary_y": True,"r":0.06}]])


In [ ]:
# Candle + Call/Put OI
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_calls(df_oi=df_oi)
trace_puts(df_oi=df_oi)
fig_show(title="Put/Call OI (" + banner + ")")

In [ ]:
# Candle + Bullish OI
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_short_calls(df_oi=df_oi)
trace_long_puts(df_oi=df_oi)
fig_show(title="Bullish OI (" + banner + ")")

In [ ]:
# Candle + Bearish OI
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_long_calls(df_oi=df_oi)
trace_short_puts(df_oi=df_oi)
fig_show(title="Bearish OI")

In [ ]:
# Candle + Call/Put OI
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_calls(df_oi=df_oi)
trace_puts(df_oi=df_oi)
trace_long_calls(df_oi=df_oi)
trace_short_puts(df_oi=df_oi)
trace_short_calls(df_oi=df_oi)
trace_long_puts(df_oi=df_oi)
fig_show(title="OI (" + banner + ")")

In [ ]:
#Net Directional OI(
df_oi = load_oi(oi_ticker)
df_ndoi = calculate_ndoi(df_oi=df_oi)

fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_ndoi(df_ndoi)
fig_show(title="Net Directional OI (NDOI) (" + banner + ")")

In [ ]:
#Call/Put Skew
df_oi = load_oi(oi_ticker)
df_ndoi = calculate_ndoi(df_oi=df_oi)

fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_call_skew(df_ndoi)
trace_put_skew(df_ndoi)
fig_show(title="Call/Put Directional Skew (NDOI) (" + banner + ")")

In [ ]:
#Bull/Bear OI
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_bull_oi(df_oi=df_oi)
trace_bear_oi(df_oi=df_oi)
fig_show(title="Bull/Bear OI (" + banner + ")")

In [ ]:
#VOEX
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_voex(df_oi=df_oi)
fig_show(title="VoEx (" + banner + ")",ytitle="VoEx")

In [ ]:
#VOEX RSI
fig_init()
trace_candle(df_ohlc=df_ohlc)
trace_voex_rsi(df_oi=df_oi)
fig_show(title="VoEx RSI (" + banner + ")",ytitle="VoEx RSI")